<a href="https://colab.research.google.com/github/kartikayladdha/NLP/blob/main/Labs/Lab3/NLP_3_Text_Classification_Consumer_Complaints_Eg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification is grouping of text to a particular category

In [ ]:
import nltk as nltk
# import gensim as gs
# import spacy as sp
# import textblob as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Consumer_Complaints_big.csv')
df.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,07/17/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,ALLY FINANCIAL INC.,TX,75035,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309495
1,07/17/2019,Credit card or prepaid card,General-purpose prepaid card,Trouble using the card,Problem using the card to withdraw money from ...,NaN,NaN,SQUARE INC,KS,674XX,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3310031
2,07/17/2019,Debt collection,Other debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,"Diversified Consultants, Inc.",FL,NaN,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309687
3,07/17/2019,Mortgage,VA mortgage,Trouble during payment process,NaN,NaN,NaN,"FLAGSTAR BANK, FSB",VA,22554,Servicemember,NaN,Web,07/17/2019,In progress,Yes,NaN,3308925
4,07/17/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes it acted appropriately as aut...,BYL Collection Services,TN,370XX,Servicemember,NaN,Web,07/17/2019,Closed with explanation,Yes,NaN,3308914


In [ ]:
df1 = df[['Product','Consumer complaint narrative']]

In [ ]:
df1['Product'].value_counts()

Mortgage                                                                        282345
Debt collection                                                                 253853
Credit reporting, credit repair services, or other personal consumer reports    252092
Credit reporting                                                                140432
Credit card                                                                      89190
Bank account or service                                                          86206
Student loan                                                                     52889
Credit card or prepaid card                                                      52468
Checking or savings account                                                      45000
Consumer Loan                                                                    31604
Vehicle loan or lease                                                            12408
Money transfer, virtual currency, or money 

We'll be combining many products now, so that we have all the classes mutually exclusive

In [ ]:
df1['Product'] = df1['Product'].str.lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1.isnull().sum()

Product                              0
Consumer complaint narrative    938347
dtype: int64

We need to remove the null values

In [ ]:
df1.shape

(1334876, 2)

In text we do not fill records, we just drop them

In [ ]:
df1= df1.dropna()

Clubbing the reporting categories so that they are mutually exclusive

In [ ]:
def renaming(x):
   #print(x)
   if x =='credit reporting':
        x = 'credit reporting, credit repair services, or other personal consumer reports'
   elif x == 'credit card':
        x = 'credit card or prepaid card'
   elif x == 'virtual currency':
        x = 'money transfer, virtual currency, or money service'
   elif x == 'prepaid card':
        x = 'credit card or prepaid card'
   elif x == 'money transfers':
        x = 'money transfer, virtual currency, or money service'
   elif x == 'student loan': 
        x='Loans'
   elif x=='consumer loan': 
        x='Loans'
   elif x=='payday loan': 
        x='Loans'
   elif x=='checking or savings account': 
        x='bank account or service'
   elif x=='payday loan, title loan, or personal loan': 
        x='Loans'
   elif x=='vehicle loan or lease': 
        x='Loans'
   return x

In [ ]:
df1['Product'] = df1['Product'].apply(renaming)

In [ ]:
df1['Product'].value_counts()

credit reporting, credit repair services, or other personal consumer reports    129418
debt collection                                                                  89379
mortgage                                                                         54109
Loans                                                                            44409
credit card or prepaid card                                                      43103
bank account or service                                                          28554
money transfer, virtual currency, or money service                                7265
other financial service                                                            292
Name: Product, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1['Product'] = le.fit_transform(df1['Product'])

In [ ]:
value_counts = df1['Product'].value_counts()
value_counts

3    129418
4     89379
6     54109
0     44409
2     43103
1     28554
5      7265
7       292
Name: Product, dtype: int64

Due to class imbalance problem, since they are very less we remove them 

In [ ]:
df1 = df1[~df1['Product'].isin(value_counts[value_counts<10000].index)]
df1['Product'].value_counts()

3    129418
4     89379
6     54109
0     44409
2     43103
1     28554
Name: Product, dtype: int64

In [ ]:
from string import punctuation
from nltk.corpus import stopwords
stuff_to_be_removed = list(stopwords.words('english'))+list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()
corpus = df1['Consumer complaint narrative'].tolist()
len(corpus)
corpus = corpus[1:10000] #for demonstration, taking less data

In [ ]:
final_corpus = []
for i in range(len(corpus)):
  word = word_tokenize(corpus[i].lower())
  word = [stemmer.stem(y) for y in word if y not in stuff_to_be_removed]
  j = " ".join(word)
  final_corpus.append(j)
#final_corpus

In [ ]:
final_corpus

['16000.00 debt',
 'cal dont answ cal cal day long hav sign contact company nev comunicacin ag nev sign contract mad commit maury cob assocy consum law credit company sign contact collect nee remov xxxx report',
 "provid account inform show debt paid credit karm stil show 2 doraget stat hospit bil 500.00 300.00 sent auth numb account paid numb 's stil credit report",
 "2 charg credit report collect ag `` trid asset man `` claim 2 pay xxxx 1 xx/xx/2015 amount 34.00 second xx/xx/2015 amount 140.00 nev xxxx account nam disput claim deny cal ag find numb fax read review expery problem 'm",
 'xxxx xxxx xxxx xxxx post 3 collect account 3 credit report company xxxx xxxx xxxx equifax on 30.00 on 100.00 on 70.00 account paid collect ag possess let xxxx xxxx xxxx dat xx/xx/2019 3 paid satisfy xxxx xxxx acct xxxx request 3 credit ag let multipl phon cal remov credit fil dat complaint xxxx xxxx xxxx delet credit fil equifax refus remov inform remain 7 year ev satisfy spok xxxx xxxx xxxx inform ele

In [ ]:
product_list=df1['Product'].tolist()
product_list=product_list[1:10000]
newdf = pd.DataFrame(final_corpus)
newdf['label'] = product_list
newdf.columns=['sentence','label']
newdf.head()

,sentence,label
0,16000.00 debt,3
1,cal dont answ cal cal day long hav sign contac...,4
2,provid account inform show debt paid credit ka...,4
3,2 charg credit report collect ag `` trid asset...,4
4,xxxx xxxx xxxx xxxx post 3 collect account 3 c...,3


Since our ML models only understand numbers, so we need to convert language into numbers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(newdf['sentence'])
x_train = vector.toarray()
print(x_train)
y_train= newdf['label']

[[0.27715548 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.20053722 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.05226989 0.         0.         ... 0.         0.         0.        ]]


NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import MultinomialNB

ab = MultinomialNB()

ab.fit(x_train,y_train)

ypred = ab.predict(x_train)

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print(accuracy_score(y_train,ypred))

print(confusion_matrix(y_train,ypred))

print(classification_report(y_train,ypred))

0.6338633863386338
[[ 260    0    3  625  113    0]
 [   0   38   64  466   34    0]
 [   0    0  263  805   43    0]
 [   0    3    0 3971   73    1]
 [   3    0    3  833 1551    1]
 [   5    0    0  540   46  255]]
              precision    recall  f1-score   support

           0       0.97      0.26      0.41      1001
           1       0.93      0.06      0.12       602
           2       0.79      0.24      0.36      1111
           3       0.55      0.98      0.70      4048
           4       0.83      0.65      0.73      2391
           6       0.99      0.30      0.46       846

    accuracy                           0.63      9999
   macro avg       0.84      0.42      0.46      9999
weighted avg       0.75      0.63      0.59      9999



RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees

classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)

classifier.fit(x_train, y_train)

ypred = classifier.predict(x_train)

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print(accuracy_score(y_train,ypred))

print(confusion_matrix(y_train,ypred))

print(classification_report(y_train,ypred))

SVM

In [ ]:
from sklearn.svm import LinearSVC

svc = LinearSVC().fit(x_train,y_train)

y_pred = svc.predict(x_train)

print(accuracy_score(y_train,y_pred))

print(confusion_matrix(y_train,y_pred))

print(classification_report(y_train,y_pred))

0.9541954195419542
[[ 934    1    6   33   27    0]
 [   0  593    7    2    0    0]
 [   1    9 1052   32   17    0]
 [  19    3   18 3891  107   10]
 [  24    3   12  101 2241   10]
 [   2    1    2   10    1  830]]
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      1001
           1       0.97      0.99      0.98       602
           2       0.96      0.95      0.95      1111
           3       0.96      0.96      0.96      4048
           4       0.94      0.94      0.94      2391
           6       0.98      0.98      0.98       846

    accuracy                           0.95      9999
   macro avg       0.96      0.96      0.96      9999
weighted avg       0.95      0.95      0.95      9999

